In [16]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [17]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [18]:
openai = OpenAI()

In [19]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [20]:
# tom = Website("https://clearchoice.io")
# print(tom.title)
# print(tom.text)


In [21]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents \
of a website to identify and describe its products and services. \
For each product or service, provide a concise summary of its core \
capabilities and features. Ignore any navigation-related text or \
irrelevant content. Present the information in a well-structured \
Markdown format, using headings and bullet points where appropriate." 

In [63]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
Include a matrix of competitive products by name that compares each products capabilities with the website. Rate \
each products capabilities compared with the website.\
Include a summary depicting the best product, also include the urls for each product \
as well as any news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [23]:
#print(user_prompt_for(tom))

In [64]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [25]:
# Try this out, and then try for a few more websites

#messages_for(tom)

In [65]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [27]:
#summarize("https://docusign.com")

In [66]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))
    # Write the summary to a text file
    website = Website(url)
    output_file = f"{website.title}.md"
    try:
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(summary)
        print(f"Summary successfully written to {output_file}")
    except IOError as e: 
        print(f"An error occurred while writing to the file: {e}")

In [67]:
display_summary("https://securustech.net")

# Securus Technologies - Friends and Family Video Visitation System

Securus Technologies provides communication solutions tailored for families of incarcerated individuals, primarily focusing on video visitation, messaging, and phone services. The platform enables users to connect with their loved ones through video calls, messaging services, and phone calls, all from the comfort of their own homes or anywhere with internet access.

## Products and Services

### Video Connect
- **Core Capability**: A fully web-based visual communication system allowing video sessions with incarcerated individuals.
- **Features**:
  - Schedule video meetings from any location with internet access.
  - Free Securus app available for easy access on mobile devices.
  - Supports calls from family, friends, attorneys, and public officials.

### eMessaging™
- **Core Capability**: A messaging platform for sharing life updates and encouragement.
- **Features**:
  - Send messages, photos, and video clips.
  - Send eCards for special occasions like birthdays and holidays.

### Phone Products
- **Core Capability**: Options to receive calls from incarcerated individuals.
- **Features**:
  - Open prepaid accounts (AdvanceConnect® accounts) for instant call receiving.
  - Various call plans available for family members.

## Competitive Product Matrix

| Product Name            | Video Calls | Messaging | Phone Services | Ease of Use | Additional Notes                 |
|------------------------|-------------|-----------|----------------|-------------|----------------------------------|
| **Securus Technologies** | Yes         | Yes       | Yes            | High        | Web-based and app available.     |
| **GTL**                | Yes         | Yes       | Yes            | Medium      | Similar pricing but varies by facility.|
| **Telmate**            | Yes         | No        | Yes            | Medium      | Focus on video visitation primarily.|
| **InmateCalling.com**  | No          | No        | Yes            | Low         | Primarily phone services, no video. |

## Best Product Summary

**Securus Technologies** stands out as the best product within the realm of communication solutions for families of incarcerated individuals. It offers a well-integrated experience with video visitation, messaging, and phone services, making it convenient for users to maintain contact with their loved ones. Its user-centric design and features like scheduling and accessibility through a mobile app set it apart from competitors.

## URLs for Products
- **Securus Technologies**: [Securus Technologies](https://www.securustech.net)
- **GTL**: [GTL](https://www.gtl.net)
- **Telmate**: [Telmate](https://www.telmate.com)
- **InmateCalling.com**: [InmateCalling.com](https://www.inmatecalling.com)

## News and Announcements Summary

- **FCC Order Impact**: Information is available regarding the 2024 FCC order affecting incarcerated communications, indicating a focus on regulatory compliance and consumer education.
- **Securus Originals**: New content such as "Day One with Andre Norman" and "Sec2nd Acts" highlights Securus’s commitment to supporting returning citizens and their families through media and outreach.

---

This comprehensive overview of Securus Technologies highlights its position in the marketplace and the specific features that make it a standout option for individuals seeking to maintain relationships with incarcerated loved ones.

Summary successfully written to Securus Technologies - Friends and Family Video Visitation System.md


In [30]:
#display_summary("https://anthropic.com")

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

PR instructions courtesy of an AI friend: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c